# Before you begin


1.   Use the [Cloud Resource Manager](https://console.cloud.google.com/cloud-resource-manager) to Create a Cloud Platform project if you do not already have one.
2.   [Enable billing](https://support.google.com/cloud/answer/6293499#enable-billing) for the project.
3.   [Enable BigQuery](https://console.cloud.google.com/flows/enableapi?apiid=bigquery) APIs for the project.


### Provide your credentials to the runtime

In [5]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


## Optional: Enable data table display

Colab includes the ``google.colab.data_table`` package that can be used to display large pandas dataframes as an interactive data table.
It can be enabled with:

In [6]:
%load_ext google.colab.data_table

If you would prefer to return to the classic Pandas dataframe display, you can disable this by running:
```python
%unload_ext google.colab.data_table
```

# Use BigQuery via magics

The `google.cloud.bigquery` library also includes a magic command which runs a query and either displays the result or saves it to a variable as a `DataFrame`.

In [8]:
# Display query output immediately

%%bigquery --project XXXyourprojectid
SELECT 
  COUNT(*) as total_rows
FROM `bigquery-public-data.samples.gsod`

,total_rows
0,114420316


In [9]:
# Save output in a variable `df`

%%bigquery --project XXXyourprojectid df
SELECT 
  COUNT(*) as total_rows
FROM `bigquery-public-data.samples.gsod`

In [10]:
df

,total_rows
0,114420316


# Use BigQuery through google-cloud-bigquery

See [BigQuery documentation](https://cloud.google.com/bigquery/docs) and [library reference documentation](https://googlecloudplatform.github.io/google-cloud-python/latest/bigquery/usage.html).

The [GSOD sample table](https://bigquery.cloud.google.com/table/bigquery-public-data:samples.gsod) contains weather information collected by NOAA, such as precipitation amounts and wind speeds from late 1929 to early 2010.


### Declare the Cloud project ID which will be used throughout this notebook

In [13]:
project_id = 'XXXyourprojectid'

### Sample approximately 2000 random rows

In [14]:
from google.cloud import bigquery

client = bigquery.Client(project=project_id)

sample_count = 2000
row_count = client.query('''
  SELECT 
    COUNT(*) as total
  FROM `bigquery-public-data.samples.gsod`''').to_dataframe().total[0]

df = client.query('''
  SELECT
    *
  FROM
    `bigquery-public-data.samples.gsod`
  WHERE RAND() < %d/%d
''' % (sample_count, row_count)).to_dataframe()

print('Full dataset has %d rows' % row_count)

Full dataset has 114420316 rows


### Describe the sampled data

In [15]:
df.describe()

,station_number,wban_number,year,month,day,mean_temp,num_mean_temp_samples,mean_dew_point,num_mean_dew_point_samples,mean_sealevel_pressure,num_mean_sealevel_pressure_samples,mean_station_pressure,num_mean_station_pressure_samples,mean_visibility,num_mean_visibility_samples,mean_wind_speed,num_mean_wind_speed_samples,max_sustained_wind_speed,max_gust_wind_speed,max_temperature,total_precipitation,snow_depth
count,2022.000000,2022.000000,2022.000000,2022.000000,2022.000000,2022.000000,2022.000000,1943.000000,1943.000000,1534.000000,1534.000000,723.000000,723.000000,1805.000000,1805.000000,1997.000000,1997.000000,1959.00000,278.000000,2020.000000,1848.000000,102.000000
mean,524447.930267,90155.403561,1987.029674,6.521266,16.121167,52.918497,13.130564,41.693155,13.030365,1014.719034,11.367014,968.777593,12.756570,12.202548,12.699169,6.737056,13.085628,12.12828,24.793885,44.220941,0.074389,10.795098
std,290933.048872,26515.654414,16.020494,3.472276,8.845289,24.089139,7.982277,22.659111,7.967080,9.011062,7.510392,67.185990,8.044643,9.565888,7.867305,4.600251,7.966185,6.56264,9.367888,24.329247,0.335241,10.639357
min,10070.000000,1.000000,1937.000000,1.000000,1.000000,-51.299999,4.000000,-59.200001,4.000000,967.000000,4.000000,622.400024,4.000000,0.000000,4.000000,0.000000,4.000000,1.00000,3.900000,-58.900002,0.000000,0.400000
25%,288972.500000,99999.000000,1977.000000,3.000000,9.000000,39.225000,7.000000,29.299999,7.000000,1009.299988,6.000000,950.700012,7.000000,6.500000,6.000000,3.500000,7.000000,7.80000,19.100000,31.950000,0.000000,2.400000
50%,568660.000000,99999.000000,1989.000000,6.000000,16.000000,55.850000,8.000000,43.599998,8.000000,1014.400024,8.000000,995.400024,8.000000,9.800000,8.000000,5.800000,8.000000,11.10000,22.900000,47.299999,0.000000,6.700000
75%,725682.500000,99999.000000,2000.000000,10.000000,24.000000,71.300003,23.000000,57.750000,23.000000,1020.099976,20.000000,1009.949982,24.000000,14.900000,23.000000,8.900000,23.000000,15.50000,28.900000,62.200001,0.010000,15.000000
max,999999.000000,99999.000000,2010.000000,12.000000,31.000000,103.000000,24.000000,82.099998,24.000000,1055.599976,24.000000,1033.199951,24.000000,99.400002,24.000000,41.599998,24.000000,61.00000,69.900002,90.300003,6.230000,49.200001


### View the first 10 rows

In [16]:
df.head(10)

,station_number,wban_number,year,month,day,mean_temp,num_mean_temp_samples,mean_dew_point,num_mean_dew_point_samples,mean_sealevel_pressure,num_mean_sealevel_pressure_samples,mean_station_pressure,num_mean_station_pressure_samples,mean_visibility,num_mean_visibility_samples,mean_wind_speed,num_mean_wind_speed_samples,max_sustained_wind_speed,max_gust_wind_speed,max_temperature,max_temperature_explicit,min_temperature,min_temperature_explicit,total_precipitation,snow_depth,fog,rain,snow,hail,thunder,tornado
0,999999,25237,1962,12,21,37.700001,8,36.000000,8.0,NaN,NaN,NaN,NaN,8.5,7.0,12.600000,8.0,32.099998,NaN,34.000000,True,None,None,NaN,NaN,True,True,True,True,True,True
1,478570,99999,1977,12,22,47.099998,15,35.599998,15.0,NaN,NaN,NaN,NaN,7.0,15.0,6.200000,15.0,11.800000,NaN,39.200001,True,None,None,0.00,NaN,False,False,False,False,False,False
2,30590,99999,1997,1,12,51.000000,12,47.000000,12.0,NaN,NaN,NaN,NaN,7.0,12.0,13.600000,12.0,23.900000,33.0,48.200001,True,None,None,0.00,NaN,False,False,False,False,False,False
3,680700,99999,2005,12,10,69.099998,5,63.900002,5.0,NaN,NaN,NaN,NaN,5.7,5.0,6.800000,5.0,8.900000,NaN,63.900002,True,None,None,0.12,NaN,False,False,False,False,False,False
4,877650,99999,1979,6,23,39.400002,16,34.900002,16.0,NaN,NaN,920.500000,15.0,21.9,16.0,16.799999,16.0,28.000000,NaN,35.599998,False,None,None,0.04,NaN,False,False,False,False,False,False
5,725060,99999,1981,1,19,26.500000,24,17.100000,24.0,NaN,NaN,998.299988,24.0,11.3,24.0,8.700000,24.0,15.900000,22.0,20.100000,True,None,None,0.00,NaN,False,False,False,False,False,False
6,714710,99999,2008,9,29,59.700001,4,41.400002,4.0,1018.700012,4.0,941.000000,4.0,NaN,NaN,7.500000,4.0,8.900000,NaN,49.099998,False,None,None,0.00,NaN,False,False,False,False,False,False
7,976860,99999,1998,3,28,71.099998,6,64.599998,6.0,1010.700012,5.0,837.200012,6.0,6.7,6.0,3.900000,6.0,12.000000,NaN,58.799999,False,None,None,0.00,NaN,False,False,False,False,False,False
8,155490,99999,1995,3,7,49.099998,6,37.700001,6.0,1007.099976,6.0,NaN,NaN,11.4,6.0,2.900000,6.0,5.800000,NaN,44.599998,True,None,None,0.00,NaN,False,False,False,False,False,False
9,165390,99999,1973,11,14,59.799999,7,53.299999,7.0,1015.400024,7.0,NaN,NaN,6.0,7.0,17.799999,7.0,20.000000,NaN,55.400002,False,None,None,0.00,NaN,False,False,False,False,False,False


In [17]:
# 10 highest total_precipitation samples
df.sort_values('total_precipitation', ascending=False).head(10)[['station_number', 'year', 'month', 'day', 'total_precipitation']]

,station_number,year,month,day,total_precipitation
693,450070,2003,5,5,6.23
93,239550,1965,12,5,5.91
1704,724080,1994,7,18,3.45
590,360200,1961,2,25,2.95
1345,279620,1965,3,18,2.95
132,296640,1970,5,19,2.95
1661,442310,1960,5,1,2.95
496,228870,1965,11,9,2.95
1612,297260,1970,12,12,2.95
997,764910,2005,9,8,2.63


# Use BigQuery through pandas-gbq

The `pandas-gbq` library is a community led project by the pandas community. It covers basic functionality, such as writing a DataFrame to BigQuery and running a query, but as a third-party library it may not handle all BigQuery features or use cases.

[Pandas GBQ Documentation](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_gbq.html)

In [18]:
import pandas as pd

sample_count = 2000
df = pd.io.gbq.read_gbq('''
  SELECT name, SUM(number) as count
  FROM `bigquery-public-data.usa_names.usa_1910_2013`
  WHERE state = 'TX'
  GROUP BY name
  ORDER BY count DESC
  LIMIT 100
''', project_id=project_id, dialect='standard')

df.head()

,name,count
0,James,272793
1,John,235139
2,Michael,225320
3,Robert,220399
4,David,219028


# Syntax highlighting
`google.colab.syntax` can be used to add syntax highlighting to any Python string literals which are used in a query later.

In [19]:
from google.colab import syntax
query = syntax.sql('''
SELECT
  COUNT(*) as total_rows
FROM
  `bigquery-public-data.samples.gsod`
''')

pd.io.gbq.read_gbq(query, project_id=project_id, dialect='standard')

,total_rows
0,114420316
